In [1]:
import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#---------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Plot_Box_sns
import GrubbsTest

# NOTES

instantaneous is old format
<br> Horizontal format with one row for each meter(?)

usage_instantaneous is new format
<br> Tall format with multiple rows for each meter

For each, the database to use is inst_msr_consume (I believe)


Also, look into meter_events.end_device_event

In [2]:
my_meter_premise_info = pd.read_csv(r'C:\Users\s346557\Documents\my_meter_premise_info.csv')
my_meter_premise_info = clm.remove_prepend_from_columns_in_df(my_meter_premise_info)
my_meter_premise_info_dict = my_meter_premise_info.squeeze().to_dict()

mp_where_str = "prem_nb = '{}' AND meter_id = '{}'".format(
    my_meter_premise_info_dict['prem_nb'], 
    my_meter_premise_info_dict['mfr_devc_ser_nbr'])

# Setup df_usage_inst

In [3]:
conn = Utilities.get_athena_prod_aws_connection()

In [4]:
cols_of_interest_usage_inst = [
    'meter_id', 
    'read_time', 
    'read_date', 
    'msr_value', 
    'msr_type'
]


sql_gen = (
"""
SELECT {}
FROM instantaneous.inst_msr_consume
WHERE opco_cd = 'oh' 
AND read_date BETWEEN '2020-10-12' AND '2020-10-13'
LIMIT 1000000
"""
)

sql_1 = sql_gen.format(','.join(cols_of_interest_usage_inst))

In [5]:
print(sql_1)


SELECT meter_id,read_time,read_date,msr_value,msr_type
FROM instantaneous.inst_msr_consume
WHERE opco_cd = 'oh' 
AND read_date BETWEEN '2020-10-12' AND '2020-10-13'
LIMIT 1000000



In [6]:
df_1 = pd.read_sql(sql_1, conn)
df_1 = clm.remove_table_aliases(df_1)

In [7]:
df_1

,meter_id,read_time,read_date,msr_value,msr_type
0,638995061,03:31:12,2020-10-12,0.160000,InstAMPsCalc
1,638995061,17:12:26,2020-10-12,0.028800,InstkW
2,638995061,17:12:26,2020-10-12,100.400002,TempF
3,638995062,03:34:05,2020-10-12,0.028800,InstkW
4,638995062,03:34:05,2020-10-12,0.240000,InstAMPsCalc
...,...,...,...,...,...
999995,876133797,03:41:29,2020-10-12,89.599998,TempF
999996,876133797,03:41:29,2020-10-12,100.000000,PowerFactorMeter
999997,876133797,03:41:29,2020-10-12,247.100006,VoltageA
999998,876133797,17:06:50,2020-10-12,1.130000,InstAMPsCalc


In [8]:
df_1.sort_values(by=['read_date', 'read_time'])

,meter_id,read_time,read_date,msr_value,msr_type
548552,761000465,03:30:12,2020-10-12,0.211200,InstkW
548553,761000465,03:30:12,2020-10-12,0.890000,InstAMPsCalc
548554,761000465,03:30:12,2020-10-12,93.199997,TempF
548555,761000465,03:30:12,2020-10-12,95.000000,PowerFactorMeter
905673,876114864,03:30:12,2020-10-12,0.278400,InstkW
...,...,...,...,...,...
657707,875530221,18:41:15,2020-10-12,1.530000,InstAMPsCalc
657708,875530221,18:41:15,2020-10-12,87.800003,TempF
657709,875530221,18:41:15,2020-10-12,237.800003,VoltageA
657481,875530166,18:41:18,2020-10-12,0.000000,InstAMPsCalc


In [9]:
df_usage_inst_2 = df_1

In [10]:
df_usage_inst_2 = clm.remove_table_aliases(df_usage_inst_2)

In [11]:
df_usage_inst_2 = df_usage_inst_2[df_usage_inst_2['msr_value'].notna()] # Don't want rows with null measurement values, I guess...
df_usage_inst_2 = df_usage_inst_2[df_usage_inst_2['msr_value']!=' '] # Don't want rows with measurement values = ' ', I guess...
df_usage_inst_2 = df_usage_inst_2[df_usage_inst_2['msr_value']!='FAULT']

In [12]:
df_usage_inst_2['msr_value'] = df_usage_inst_2['msr_value'].astype(float)

In [13]:
msr_types = df_usage_inst_2['msr_type'].unique().tolist()

In [14]:
msr_types

['InstAMPsCalc', 'InstkW', 'TempF', 'PowerFactorMeter', 'VoltageA']

In [15]:
# measurement_types = df_usage_inst_2['measurement_type'].unique().tolist()
# for measurement_type in measurement_types:
#     tmp_df = df_usage_inst_2[df_usage_inst_2['measurement_type']==measurement_type]
#     fig, ax = plt.subplots(1, 1, num=fig_num, figsize=[11, 8.5])
#     fig.suptitle(f'Measurement Type = {measurement_type}', fontsize=25, fontweight='bold')
#     sns.boxplot(ax=ax, x='read_type', y='measurement_value', data=tmp_df)
#     ax.tick_params(axis='x', labelrotation=90, labelsize=7.0, direction='in');
#     fig_num +=1

In [16]:
df_usage_inst_2

,meter_id,read_time,read_date,msr_value,msr_type
0,638995061,03:31:12,2020-10-12,0.160000,InstAMPsCalc
1,638995061,17:12:26,2020-10-12,0.028800,InstkW
2,638995061,17:12:26,2020-10-12,100.400002,TempF
3,638995062,03:34:05,2020-10-12,0.028800,InstkW
4,638995062,03:34:05,2020-10-12,0.240000,InstAMPsCalc
...,...,...,...,...,...
999995,876133797,03:41:29,2020-10-12,89.599998,TempF
999996,876133797,03:41:29,2020-10-12,100.000000,PowerFactorMeter
999997,876133797,03:41:29,2020-10-12,247.100006,VoltageA
999998,876133797,17:06:50,2020-10-12,1.130000,InstAMPsCalc


In [17]:
df_usage_inst_2.groupby('msr_type')['msr_value'].mean()

msr_type
InstAMPsCalc          3.755363
InstkW                0.901741
PowerFactorMeter     92.272218
TempF                90.428892
VoltageA            241.832109
Name: msr_value, dtype: float64

In [18]:
df_usage_inst_2[df_usage_inst_2['msr_type']=='VoltageA']

,meter_id,read_time,read_date,msr_value,msr_type
7,638995062,03:34:05,2020-10-12,119.699997,VoltageA
12,638995062,17:11:26,2020-10-12,119.900002,VoltageA
19,638995065,03:30:51,2020-10-12,123.599998,VoltageA
25,638995066,03:30:38,2020-10-12,123.699997,VoltageA
31,638995067,17:09:39,2020-10-12,123.199997,VoltageA
...,...,...,...,...,...
999976,876133793,17:04:57,2020-10-12,247.800003,VoltageA
999979,876133794,03:38:38,2020-10-12,248.300003,VoltageA
999989,876133795,17:04:46,2020-10-12,247.199997,VoltageA
999993,876133796,17:04:44,2020-10-12,247.100006,VoltageA


In [19]:
df_usage_inst_2['meter_id'].nunique()

199866

In [20]:
df_usage_inst_2[df_usage_inst_2['meter_id']=='638995062']

,meter_id,read_time,read_date,msr_value,msr_type
3,638995062,03:34:05,2020-10-12,0.028800,InstkW
4,638995062,03:34:05,2020-10-12,0.240000,InstAMPsCalc
5,638995062,03:34:05,2020-10-12,98.599998,TempF
6,638995062,03:34:05,2020-10-12,100.000000,PowerFactorMeter
7,638995062,03:34:05,2020-10-12,119.699997,VoltageA
8,638995062,17:11:26,2020-10-12,0.028800,InstkW
9,638995062,17:11:26,2020-10-12,0.240000,InstAMPsCalc
10,638995062,17:11:26,2020-10-12,100.000000,PowerFactorMeter
11,638995062,17:11:26,2020-10-12,100.400002,TempF
12,638995062,17:11:26,2020-10-12,119.900002,VoltageA


In [21]:
msr_types

['InstAMPsCalc', 'InstkW', 'TempF', 'PowerFactorMeter', 'VoltageA']